<a href="https://colab.research.google.com/github/tackulus/229352_KAGGLE/blob/New/229352_KAGGLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
from google.colab import drive
drive.mount('/content/drive')

!unzip /content/drive/MyDrive/ECG/ECG.zip;

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWjTWZMEBQZBQXxhDd9aF-J83kKqskIlhajS-Zz8i9Aj8fjLk6r7DKA


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
X_data = np.load('/content/ECG_X_train.npy')
X_data.shape

(14630, 1000, 12)

In [ ]:
y_data = np.load('/content/ECG_y_train.npy')
y_data.shape

(14630,)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = \
    train_test_split(X_data, y_data, test_size=.25, random_state=177)

In [ ]:
# from tensorflow.keras.utils import to_categorical

# y_train = to_categorical(y_train)
# y_valid = to_categorical(y_valid)

In [ ]:
from tensorflow.keras.preprocessing import timeseries_dataset_from_array

def make_dataset(data, target, batch_size=64):
    dataset = timeseries_dataset_from_array(
        data=data,
        targets=target,
        sequence_length=1,
        sequence_stride=1,
        shuffle=False,
        batch_size=batch_size,
    )

    X, y = next(iter(dataset))
    print(">>", X.shape, y.shape)
    return dataset

train_batch = make_dataset(X_train, y_train)
valid_batch = make_dataset(X_valid, y_valid)

>> (64, 1, 1000, 12) (64, 5)
>> (64, 1, 1000, 12) (64, 5)




```
# This is formatted as code
```

## Convolutional neural network

> https://towardsdatascience.com/time-series-classification-with-deep-learning-d238f0147d6f

### submission04 acc=0.75639

In [ ]:
import tensorflow as tf
from tensorflow import keras 

def create_cnn_model(activation='relu',dropout=0.25,optimizer='adam',learn_rate=0.002):
    np.random.seed(1)
    tf.random.set_seed(1)
    model = None
    model = keras.Sequential()

    model.add(keras.layers.Conv2D(128, 5, input_shape=(X_train.shape[1], X_train.shape[2], 1), activation=activation))
    model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.MaxPool2D())

    model.add(keras.layers.Conv2D(64, 5, activation=activation,padding='same'))
    model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.MaxPool2D())

    model.add(keras.layers.Conv2D(64, 5, activation=activation,padding='same'))
    model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.MaxPool2D())

    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(128, activation=activation))
    model.add(keras.layers.Dropout(dropout))

    model.add(keras.layers.Dense(64, activation=activation))
    model.add(keras.layers.Dropout(dropout))

    model.add(keras.layers.Dense(5, activation='softmax'))

    optimizer = tf.keras.optimizers.Adam(lr=learn_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    model.summary()

    return model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
earlystopping = EarlyStopping(monitor='val_accuracy',
                              patience=10,
                              restore_best_weights=True)

model_cnn = create_cnn_model(activation='relu',dropout=0.25,optimizer='adam',learn_rate=0.001)
history = model_cnn.fit(X_train[:,:,:,None], y_train, epochs=100, validation_data=(X_valid[:,:,:,None],y_valid),batch_size=80,verbose=1,callbacks=[earlystopping])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 996, 8, 128)       3328      
_________________________________________________________________
dropout_30 (Dropout)         (None, 996, 8, 128)       0         
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 498, 4, 128)       0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 498, 4, 64)        204864    
_________________________________________________________________
dropout_31 (Dropout)         (None, 498, 4, 64)        0         
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 249, 2, 64)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 249, 2, 64)       

In [ ]:
model_cnn.evaluate(X_valid[:,:,:,None],y_valid,verbose=0)[1]

0.7714598178863525

In [ ]:
X_test = np.load('/content/ECG_X_test.npy')

preds = model_cnn.predict(X_test[:,:,:,None])
y_test = np.argmax(preds, axis=-1)
y_test = pd.DataFrame({'Class': y_test})
y_test.insert(0, 'Patient_ID', y_test.index)
y_test.shape

(1642, 2)

In [ ]:
y_test.to_csv('y_test.csv', index=False)
model_cnn.save('submission04.h5')